In [ ]:
library(tidyverse)
library(lubridate)

In [ ]:
# Physician WDR/WIS performance on the dataset of this experiment (get it from Eval DQN script)
train_phy_WDR = 2.2539
train_phy_WIS = 1.8684
val_phy_WDR   = 2.0796
val_phy_WIS   = 1.8388
test_phy_WDR  = 3.7183
test_phy_WIS  = 1.8067
exp_name = 'ess_b32_h128_lr1e4_g99'
#model_performance_name = 'performance/lucspace_1_performance.csv'

model_performance_name = paste0('../performance/' ,exp_name,'_1_performance.csv')
modelconfigfile = paste0('../models/' , exp_name , '_1_model_config.csv')
finalmodelfile = paste0( '../models/' , exp_name , '_1_model.chk')
tracking_model_performance_name = paste0('../performance/' , exp_name , '_1_tracking_performance.csv')
resultsdir = 'results/'
figuredir = 'figures/'
#location of file where we have dataframe
commondir = '../data/'
Experiment = 'Extended State-Space'


In [ ]:
library(repr) # needed for figure size
options(repr.plot.width=6, repr.plot.height=6)

########################
### MULTIPLOT
# SOURCE: http://www.cookbook-r.com/Graphs/Multiple_graphs_on_one_page_(ggplot2)/
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL) {
  library(grid)

  # Make a list from the ... arguments and plotlist
  plots <- c(list(...), plotlist)

  numPlots = length(plots)

  # If layout is NULL, then use 'cols' to determine layout
  if (is.null(layout)) {
    # Make the panel
    # ncol: Number of columns of plots
    # nrow: Number of rows needed, calculated from # of cols
    layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                    ncol = cols, nrow = ceiling(numPlots/cols))
  }

 if (numPlots==1) {
    print(plots[[1]])

  } else {
    # Set up the page
    grid.newpage()
    pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))

    # Make each plot, in the correct location
    for (i in 1:numPlots) {
      # Get the i,j matrix positions of the regions that contain this subplot
      matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE))

      print(plots[[i]], vp = viewport(layout.pos.row = matchidx$row,
                                      layout.pos.col = matchidx$col))
    }
  }
}

In [ ]:
model_performance_cols = cols(avg_best_Q = col_double(),
                                  avg_current_Q = col_double(),
                                  loss = col_double(),
                                  epoch_action_prob = col_double())
model_performance <- read_csv(paste0(model_performance_name),col_types = model_performance_cols)

In [ ]:
head(model_performance)

In [ ]:
    # Training Metric plots for experiment
avg_best_Q = model_performance %>% ggplot(aes(x=c(1:nrow(model_performance))/1000,y = avg_best_Q)) + stat_smooth(method = 'gam', formula = y ~ s(x, bs = "cs"), span=0.1, se=T, size=1) +
                                       ylim(0,NA) +
                                       labs(subtitletitle="Average State Value",title="",y="Qvalue",x="iterations", caption = '') + 
                                       theme(axis.text=element_text(size=8), axis.title=element_text(size=10)) + 
                                       theme(plot.title = element_text(size=12)) + theme(legend.position="none")


In [ ]:
avg_current_Q = model_performance %>% ggplot(aes(x=c(1:nrow(model_performance))/1000,y = avg_current_Q)) +
                                      stat_smooth(method = 'gam', formula = y ~ s(x, bs = "cs"), span=0.1, se=T, size=1) +
                                       ylim(0,NA) +
                                       labs(subtitletitle="Average Physician Action Value",title="",y="Qvalue",x="iterations", caption = '') + 
                                       theme(axis.text=element_text(size=8), axis.title=element_text(size=10)) + 
                                       theme(plot.title = element_text(size=12)) + theme(legend.position="none")



In [ ]:
loss = model_performance %>% ggplot(aes(x=c(1:nrow(model_performance))/1000,y = loss)) + stat_smooth(method = 'gam', formula = y ~ s(x, bs = "cs"), span=0.1, se=T, size=1) +
                                       ylim(0,NA) +
                                       labs(subtitletitle="Loss",title="",y="loss",x="iterations", caption = paste0("Experiment: ",as.character(Experiment))) + 
                                       theme(axis.text=element_text(size=8), axis.title=element_text(size=10)) + theme(plot.title = element_text(size=12))


In [ ]:
epoch_action_prob = model_performance %>% ggplot(aes(x=c(1:nrow(model_performance))/1000,y = epoch_action_prob*100)) + stat_smooth(method = 'gam', formula = y ~ s(x, bs = "cs"), span=0.1, se=T, size=1) +
                                       ylim(0,NA) +
                                       labs(subtitletitle="Average best action probability",title="",y="probability",x="iterations", caption = paste0("Experiment: ",as.character(Experiment))) + 
                                       theme(axis.text=element_text(size=8), axis.title=element_text(size=10)) + theme(plot.title = element_text(size=12)) + theme(legend.position="none")


In [ ]:
epoch_error = model_performance %>% ggplot(aes(x=c(1:nrow(model_performance))/1000,y = epoch_error)) + stat_smooth(method = 'gam', formula = y ~ s(x, bs = "cs"), span=0.1, se=T, size=1) +
                                       ylim(0,NA) +
                                       labs(subtitletitle="Batch error",title="",x="iteration",y="error", caption = paste0("Experiment: ",as.character(Experiment))) + 
                                       theme(axis.text=element_text(size=8), axis.title=element_text(size=10)) + theme(plot.title = element_text(size=12)) + theme(legend.position="none")


In [ ]:
epoch_td_error = model_performance %>% ggplot(aes(x=c(1:nrow(model_performance))/1000,y = epoch_td_error)) + stat_smooth(method = 'gam', formula = y ~ s(x, bs = "cs"), span=0.1, se=T, size=1) +
                                       ylim(0,NA) +
                                       labs(subtitletitle="Batch TD error",title="",y="TD error",x="iterations", caption = paste0("Experiment: ",as.character(Experiment))) + 
                                       theme(axis.text=element_text(size=8), axis.title=element_text(size=10)) + theme(plot.title = element_text(size=12)) + theme(legend.position="none")


In [ ]:
epoch_per_error = model_performance %>% ggplot(aes(x=c(1:nrow(model_performance))/1000,y = epoch_per_error)) + stat_smooth(method = 'gam', formula = y ~ s(x, bs = "cs"), span=0.1, se=T, size=1) +
                                       ylim(0,NA) +
                                       labs(subtitletitle="Batch PER error",title="",x="iteration",y="PER error", caption = paste0("Experiment: ",as.character(Experiment))) + 
                                       theme(axis.text=element_text(size=8), axis.title=element_text(size=10)) + theme(plot.title = element_text(size=12)) + theme(legend.position="none")


In [ ]:
epoch_reg_term = model_performance %>% ggplot(aes(x=c(1:nrow(model_performance))/1000,y = epoch_reg_term)) + geom_line() +
                                       labs(subtitletitle="Batch regularization term",title="",y="reg term",x="iterations", caption = paste0("Experiment: ",as.character(Experiment))) + 
                                       theme(axis.text=element_text(size=8), axis.title=element_text(size=10)) + theme(plot.title = element_text(size=12)) + theme(legend.position="none")



In [ ]:
    options(repr.plot.width=10, repr.plot.height=5)
    suppressWarnings(multiplot(avg_best_Q, avg_current_Q, loss, epoch_action_prob, epoch_error, epoch_td_error, epoch_per_error, epoch_reg_term, cols=4))
    #tiff(filename = paste('figures/tracking_multiplot_expgrid_', Experiment, '.tiff', sep=""), width = 10, height = 5, units = "in", res = 200, pointsize=12)
    #suppressWarnings(multiplot(avg_best_Q, avg_current_Q, loss, epoch_action_prob, epoch_error, epoch_td_error, epoch_per_error, epoch_reg_term, cols=4))
    #dev.off()

In [ ]:
dev.off()

#### Model WDR / WIS performance PLOTS

In [ ]:
### Load tracking performance data 
tracking_performance_cols = cols(eval_type = col_character(),
                                     iteration = col_integer(),
                                     model_WDR = col_double(),
                                     model_wis = col_double())
tracking_performance <- read_csv(paste0(tracking_model_performance_name), col_types = tracking_performance_cols)
head(tracking_performance)

In [ ]:
ylim_min_WDR = floor(min(tracking_performance$model_WDR))-1
ylim_min_WIS = floor(min(tracking_performance$model_wis))-1
ylim_max_WDR = ceiling(max(tracking_performance$model_WDR))+1
ylim_max_WIS = ceiling(max(tracking_performance$model_wis))+1

In [ ]:
train_tracking_performance = tracking_performance[tracking_performance$eval_type == 'train', ]
val_tracking_performance = tracking_performance[tracking_performance$eval_type == 'val', ]
test_tracking_performance = tracking_performance[tracking_performance$eval_type == 'test', ]

In [ ]:
train_WIS = train_tracking_performance %>%  select(iteration, model_wis) %>% 
                                    gather(key,value, model_wis) %>% 
                                    ggplot(aes(x=iteration, y = value, colour=key)) + geom_line(colour="green") + geom_smooth(method="loess", se=F, colour="green") + 
                                           scale_colour_discrete(name="Policy", labels=c("Physician", "DDQN model")) +
                                    geom_hline(yintercept=test_phy_WIS, color = "blue") +
                                    ylim(ylim_min_WIS,ylim_max_WIS) +
                                    labs(subtitle="TRAIN - WIS",y="WIS",x="iterations", 
                                    caption = paste0("Experiment: ",as.character(Experiment))) +  theme(legend.position="none")


In [ ]:
train_WDR = train_tracking_performance %>%  select(iteration, model_WDR) %>% 
                                    gather(key,value, model_WDR) %>% 
                                    ggplot(aes(x=iteration, y = value, colour=key)) + geom_line() + geom_smooth(method="loess", se=F) + 
                                           scale_colour_discrete(name="Policy", breaks=c("Phys_WDR", "model_WDR"), labels=c("Physician", "DDQN model")) +
                                    geom_hline(yintercept=train_phy_WDR, color = "blue") +
                                    ylim(ylim_min_WDR,ylim_max_WDR) +
                                    labs(subtitle="TRAIN - WDR",y="WDR",x="iterations", 
                                    caption = paste0("Experiment: ",as.character(Experiment))) +  theme(legend.position="center")

In [ ]:
    val_WIS = val_tracking_performance %>%  select(iteration, model_wis) %>% 
                                    gather(key,value, model_wis) %>% 
                                    ggplot(aes(x=iteration, y = value, colour=key)) + geom_line(colour="green") + geom_smooth(method="loess", se=F, colour="green") + 
                                           scale_colour_discrete(name="Policy", labels=c("Physician", "DDQN model")) +
                                    geom_hline(yintercept=test_phy_WIS, color = "blue") +
                                    ylim(ylim_min_WIS,ylim_max_WIS) +
                                    labs(subtitle="VAL - WIS",y="WIS",x="iterations", 
                                    caption = paste0("Experiment: ",as.character(Experiment))) +  theme(legend.position="none")

    val_WDR = val_tracking_performance %>%  select(iteration, model_WDR) %>% 
                                    gather(key,value, model_WDR) %>% 
                                    ggplot(aes(x=iteration, y = value, colour=key)) + geom_line() + geom_smooth(method="loess", se=F) + 
                                           scale_colour_discrete(name="Policy", breaks=c("Phys_WDR", "model_WDR"), labels=c("Physician", "DQN model")) +
                                    geom_hline(yintercept=val_phy_WDR, color = "blue") +
                                    ylim(ylim_min_WDR,ylim_max_WDR) +
                                    labs(subtitle="VAL - WDR",y="WDR",x="iterations", 
                                    caption =paste0("Experiment: ",as.character(Experiment))) +  theme(legend.position="none")

    test_WIS = test_tracking_performance %>%  select(iteration, model_wis) %>% 
                                    gather(key,value, model_wis) %>% 
                                    ggplot(aes(x=iteration, y = value, colour=key)) + geom_line(colour="green") + geom_smooth(method="loess", se=F, colour="green") + 
                                           scale_colour_discrete(name="Policy", labels=c("Physician", "DDQN model")) +
                                    geom_hline(yintercept=test_phy_WIS, color = "blue") +
                                    ylim(ylim_min_WIS,ylim_max_WIS) +
                                    labs(subtitle="TEST - WIS",y="WIS",x="iterations", 
                                    caption = paste0("Experiment: ",as.character(Experiment))) +  theme(legend.position="none")

    test_WDR = test_tracking_performance %>%  select(iteration, model_WDR) %>%  
                                    gather(key,value, model_WDR) %>% 
                                    ggplot(aes(x=iteration, y = value, colour=key)) + geom_line() + geom_smooth(method="loess", se=F) + 
                                           scale_colour_discrete(name="Policy", breaks=c("Phys_WDR", "model_WDR"), labels=c("Physician", "DQN model")) +
                                     theme(legend.title = element_text(colour="blue", size=20, face="bold")) +
                                     theme(legend.text = element_text(size = 16)) + 
                                    geom_hline(yintercept=test_phy_WDR, color = "blue") +
                                    ylim(ylim_min_WDR,ylim_max_WDR) +
                                    labs(subtitle="TEST - WDR",y="WDR",x="iterations", 
                                    caption = paste0("Experiment: ",as.character(Experiment))) +  theme(legend.position = "none")

In [ ]:
options(repr.plot.width=12, repr.plot.height=4)
suppressWarnings(multiplot(train_WIS, train_WDR, val_WIS, val_WDR, test_WIS, test_WDR, cols=3))

In [ ]:
PHY = as.data.frame(matrix(ncol=0,nrow=1))
PHY$WDR_train = train_phy_WDR
PHY$WDR_val   = val_phy_WDR
PHY$WDR_test  = test_phy_WDR
PHY$WIS_train = train_phy_WIS
PHY$WIS_val   = val_phy_WIS
PHY$WIS_test  = test_phy_WIS
PHY$exp_grid  = "Physician"
PHY$iteration = "Physician"
results = PHY
print(results)

In [ ]:
head(tracking_performance)

In [ ]:
      ### LOAD DATA
    tracking_performance_cols = cols(eval_type = col_character(),
                                     iteration = col_integer(),
                                     model_WDR = col_double(),
                                     model_wis = col_double())
    tracking_performance <- read_csv(paste0(tracking_model_performance_name), col_types = tracking_performance_cols)

    ########################
    WDR = tracking_performance %>% select(iteration,eval_type, model_WDR) %>% spread(eval_type, model_WDR) %>% rename(WDR_test=test,WDR_train=train,WDR_val=val)
    WIS = tracking_performance %>% select(iteration,eval_type, model_wis) %>% spread(eval_type, model_wis) %>% rename(WIS_test=test,WIS_train=train,WIS_val=val) %>% select(-iteration)
    tpdf = cbind(WDR,WIS)

In [ ]:
    tpdf$train_phy_WDR = train_phy_WDR
    tpdf$train_phy_WIS = train_phy_WIS
    tpdf$val_phy_WDR   = val_phy_WDR
    tpdf$val_phy_WIS   = val_phy_WIS
    tpdf$test_phy_WDR  = test_phy_WDR
    tpdf$test_phy_WIS  = test_phy_WIS
    
    
    # select models with good performance on all datasets
    
    result = tpdf[ (
              (tpdf$WDR_train>1.0*tpdf$train_phy_WDR)
              &
              (tpdf$WIS_train>1.0*tpdf$train_phy_WIS)
              &
              (tpdf$WDR_val>1.0*tpdf$val_phy_WDR)
              &
              (tpdf$WIS_val>1.0*tpdf$val_phy_WIS)
              &
              (tpdf$WDR_test>1.0*tpdf$test_phy_WDR)
              &
              (tpdf$WIS_test>1.0*tpdf$test_phy_WIS)
          )
         , ]
        

In [ ]:
resultsdir = '../results/'

In [ ]:
    exp_grid=1
    # collect model results
    if(nrow(result)>=1){ 
        result$exp_grid = exp_grid
        result = result[order(-result$WDR_val, -result$WIS_val),]
        result <- result[c("WDR_train", "WDR_val", "WDR_test", "WIS_train", "WIS_val", "WIS_test", "exp_grid", "iteration")]
        print(head(result))
    } else { 
        print("No good models") 
    }
    results = rbind(results,result)



### SAVE AND SHOW RESULTS

# order results by validation WDR and WIS (best values up top)
results = results[order(-results$WDR_val, -results$WIS_val),]
# re-order again to bring physician returns back up top
results = results[c(nrow(results),1:nrow(results)-1),]

if(nrow(results)>=1){ 
    # Write CSV
    experiment_results = paste0(resultsdir, exp_name, '_results.csv', sep="")
    write.csv(results, file = experiment_results)
}

# show results
print(experiment_results)
#results[(results$iteration!=0), ]